In [1]:
import numpy  as np 
import pandas as pd 

In [2]:
test = pd.read_csv('../data/test_data.csv', )

# preview:
test.head(5)

,home_avg_odds_hr_0,home_avg_odds_hr_1,home_avg_odds_hr_2,home_avg_odds_hr_3,home_avg_odds_hr_4,home_avg_odds_hr_5,home_avg_odds_hr_6,home_avg_odds_hr_7,home_avg_odds_hr_8,home_avg_odds_hr_9,...,away_avg_odds_hr_66,away_avg_odds_hr_67,away_avg_odds_hr_68,away_avg_odds_hr_69,away_avg_odds_hr_70,away_avg_odds_hr_71,winner,favorite,has_favorite_for_winner,closely_contested
0,110.0,-110.0,100.0,110.0,110.0,110.0,110.0,110.0,110.0,110.0,...,190.0,190.0,190.0,190.0,190.0,190.0,away,home,False,True
1,-200.0,-200.0,-200.0,-200.0,-190.0,-160.0,-150.0,-170.0,-200.0,-200.0,...,270.0,290.0,280.0,270.0,270.0,280.0,home,home,True,False
2,160.0,140.0,140.0,140.0,140.0,140.0,140.0,140.0,140.0,140.0,...,150.0,150.0,150.0,150.0,140.0,140.0,away,away,False,True
3,-190.0,-190.0,-180.0,-180.0,-170.0,-170.0,-170.0,-170.0,-180.0,-190.0,...,430.0,400.0,420.0,420.0,420.0,400.0,home,home,True,False
4,-140.0,-140.0,-140.0,-140.0,-140.0,-140.0,-150.0,-140.0,-140.0,-140.0,...,400.0,400.0,400.0,400.0,400.0,400.0,home,home,True,False


In [3]:
# generate strategy selections for test data:
import random as rd 
rd.seed(229)

picks = []
for i in range(len(test)):
    picks.append(rd.choice(['home', 'away']))

# add the picks column to the dataframe:
test['strategy_picks'] = picks

# data preview:
test.head(3)

,home_avg_odds_hr_0,home_avg_odds_hr_1,home_avg_odds_hr_2,home_avg_odds_hr_3,home_avg_odds_hr_4,home_avg_odds_hr_5,home_avg_odds_hr_6,home_avg_odds_hr_7,home_avg_odds_hr_8,home_avg_odds_hr_9,...,away_avg_odds_hr_67,away_avg_odds_hr_68,away_avg_odds_hr_69,away_avg_odds_hr_70,away_avg_odds_hr_71,winner,favorite,has_favorite_for_winner,closely_contested,strategy_picks
0,110.0,-110.0,100.0,110.0,110.0,110.0,110.0,110.0,110.0,110.0,...,190.0,190.0,190.0,190.0,190.0,away,home,False,True,home
1,-200.0,-200.0,-200.0,-200.0,-190.0,-160.0,-150.0,-170.0,-200.0,-200.0,...,290.0,280.0,270.0,270.0,280.0,home,home,True,False,home
2,160.0,140.0,140.0,140.0,140.0,140.0,140.0,140.0,140.0,140.0,...,150.0,150.0,150.0,140.0,140.0,away,away,False,True,away


In [4]:
# drop any unnecessary columns: 
test = test[['home_avg_odds_hr_71', 'away_avg_odds_hr_71', 'winner', 'favorite', 'has_favorite_for_winner', 'closely_contested', 'strategy_picks']]

# create a winner_avg_odds column: 
test['winner_avg_odds_hr_71'] = np.where(
    test['winner'] == 'home', test['home_avg_odds_hr_71'], test['away_avg_odds_hr_71']
)

# create 'relative_underdog' column: 
test['relative_underdog'] = np.where(
    test['favorite'] == 'home', 'away', 'home'
)

# preview:
test.head(3)

,home_avg_odds_hr_71,away_avg_odds_hr_71,winner,favorite,has_favorite_for_winner,closely_contested,strategy_picks,winner_avg_odds_hr_71,relative_underdog
0,110.0,190.0,away,home,False,True,home,190.0,away
1,-140.0,280.0,home,home,True,False,home,-140.0,away
2,150.0,140.0,away,away,False,True,away,140.0,home


In [11]:
N_PROFITABLE_BETS = 0
N_TOTAL_BETS      = 0

NET_VALUE_OF_BETS_WON  = []
NET_VALUE_OF_BETS_LOST = []

UNDERDOG_BETS_HITS  = []
N_UNDERDOG_BETS_TOTAL = 0

CLEAR_WINNER_BETS_HITS = []
N_CLEAR_WINNER_BETS_TOTAL = 0

CLOSELY_CONTESTED_BETS_HITS = []
N_CLOSELY_CONTESTED_BETS_TOTAL = 0

balance = 1000

# iterate through dataframe: 
for row in test.itertuples(index=False):
    
    # if there is balance remaining: 
    if balance >= 0: 
        
        # extract elements: 
        final_home_odds, final_away_odds, winner, favorite, has_favorite_for_winner, \
            closely_contested, strategy_pick, final_winner_odds, relative_underdog = row
        
        # update counts:
        if has_favorite_for_winner: N_CLEAR_WINNER_BETS_TOTAL += 1
        if closely_contested: N_CLOSELY_CONTESTED_BETS_TOTAL += 1
        
        # if strategy bet on underdog: 
        if strategy_pick == relative_underdog: 
            N_UNDERDOG_BETS_TOTAL += 1
        
        # if pick was correct: 
        if winner == strategy_pick:
            N_PROFITABLE_BETS += 1
                
            if final_winner_odds >= 0: 
                NET_VALUE_OF_BETS_WON.append(final_winner_odds)
            if final_winner_odds < 0:
                NET_VALUE_OF_BETS_WON.append(100)
            
            # if strategy correctly bet on underdog:
            if strategy_pick == relative_underdog:
                if final_winner_odds >= 0: 
                    UNDERDOG_BETS_HITS.append(final_winner_odds)
                if final_winner_odds < 0:
                    UNDERDOG_BETS_HITS.append(100)
                    
            # if the strategy correctly bet in a 'clear winner' match:
            if has_favorite_for_winner:
                if final_winner_odds >= 0: 
                    CLEAR_WINNER_BETS_HITS.append(final_winner_odds)
                if final_winner_odds < 0:
                    CLEAR_WINNER_BETS_HITS.append(100)
                    
            # if the strategy correctly bet in a 'closely contested' match:
            if closely_contested: 
                if final_winner_odds >= 0: 
                    CLOSELY_CONTESTED_BETS_HITS.append(final_winner_odds)
                if final_winner_odds < 0:
                    CLOSELY_CONTESTED_BETS_HITS.append(100)
        
            # update balance: 
            if final_winner_odds >= 0: 
                balance += final_winner_odds
            if final_winner_odds < 0:
                balance += 100
        
        # if the pick was incorrect:
        elif winner != strategy_pick:
            if final_winner_odds >= 0: 
                NET_VALUE_OF_BETS_LOST.append(100)
            if final_winner_odds < 0:
                NET_VALUE_OF_BETS_LOST.append(abs(final_winner_odds))
                
            # update balance: 
            if final_winner_odds >= 0: 
                balance -= 100
            if final_winner_odds < 0:
                balance -= abs(final_winner_odds)
                
        # update total bet count:
        N_TOTAL_BETS += 1

In [12]:
profitable_bet_rate       = N_PROFITABLE_BETS / N_TOTAL_BETS
overall_profit            = sum(NET_VALUE_OF_BETS_WON) - sum(NET_VALUE_OF_BETS_LOST)
largest_amt_won           = max(NET_VALUE_OF_BETS_WON)
smallest_amt_won          = min(NET_VALUE_OF_BETS_WON)
avg_amt_won               = sum(NET_VALUE_OF_BETS_WON) / len(NET_VALUE_OF_BETS_WON)
avg_amt_loss              = sum(NET_VALUE_OF_BETS_LOST) / len(NET_VALUE_OF_BETS_LOST)

underdog_bet_hr           = len(UNDERDOG_BETS_HITS) / N_UNDERDOG_BETS_TOTAL  if (N_UNDERDOG_BETS_TOTAL > 0) else 'no underdog bets made.'
avg_underdog_hit_winnings = sum(UNDERDOG_BETS_HITS) / len(UNDERDOG_BETS_HITS) if len(UNDERDOG_BETS_HITS) > 0 else 'no winning underdog bets made.'

clearwinner_bet_hr           = len(CLEAR_WINNER_BETS_HITS) / N_CLEAR_WINNER_BETS_TOTAL  if (N_CLEAR_WINNER_BETS_TOTAL > 0) else 'no clear favorite bets made.'
avg_clearwinner_hit_winnings = sum(CLEAR_WINNER_BETS_HITS) / len(CLEAR_WINNER_BETS_HITS) if len(CLEAR_WINNER_BETS_HITS) > 0 else 'no winning clear favorite bets made.'

closelycontested_bet_hr           = len(CLOSELY_CONTESTED_BETS_HITS) / N_CLOSELY_CONTESTED_BETS_TOTAL  if (N_CLOSELY_CONTESTED_BETS_TOTAL > 0) else 'no closely contested bets made.'
avg_closelycontested_hit_winnings = sum(CLOSELY_CONTESTED_BETS_HITS) / len(CLOSELY_CONTESTED_BETS_HITS) if len(CLOSELY_CONTESTED_BETS_HITS) > 0 else 'no winning closely contested bets made.'


metrics = {
    'profitable_bet_rate' : f'{profitable_bet_rate:.2f} ({N_PROFITABLE_BETS} / {N_TOTAL_BETS})',
    'overall_profit' : overall_profit,
    'largest_amt_won' : largest_amt_won,
    'smallest_amt_won' : smallest_amt_won,
    'avg_amt_won' : avg_amt_won,
    'avg_amt_loss' : avg_amt_loss,
    'underdog_bet_hr' : underdog_bet_hr, 
    'avg_underdog_hit_winnings' : avg_underdog_hit_winnings, 
    'clearwinner_bet_hr' : clearwinner_bet_hr,
    'avg_clearwinner_hit_winnings' : avg_clearwinner_hit_winnings,
    'closelycontested_bet_hr' : closelycontested_bet_hr,
    'avg_closelycontested_hit_winnings' : avg_closelycontested_hit_winnings
}

# report metrics
for metric in metrics.keys():    
    if type(metrics[metric]) is str: 
        print(f'{metric} :: {metrics[metric]}')
    else: 
        print(f'{metric} :: {metrics[metric]:.2f}')


profitable_bet_rate :: 0.55 (69 / 125)
overall_profit :: -7980.00
largest_amt_won :: 750.00
smallest_amt_won :: 100.00
avg_amt_won :: 166.96
avg_amt_loss :: 348.21
underdog_bet_hr :: 0.44
avg_underdog_hit_winnings :: 257.41
clearwinner_bet_hr :: 0.57
avg_clearwinner_hit_winnings :: 177.50
closelycontested_bet_hr :: 0.57
avg_closelycontested_hit_winnings :: 156.25
